In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import seaborn as sns

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('universal_tagset')
#nltk.download('vader_lexicon')

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer.model_max_length = 512

skyscanner = pd.read_csv("skyskanner.csv")

skyscanner = skyscanner.dropna(subset=['review'])
shortStringCondition = skyscanner['review'].str.len() > 50 #Keeping only the 50 char + strings
skyscanner_long = skyscanner[shortStringCondition]
# Create empty columns to store the outputs
skyscanner_long['roberta_neg'] = ""
skyscanner_long['roberta_neu'] = ""
skyscanner_long['roberta_pos'] = ""



/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skyscanner_long['roberta_neg'] = ""
/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skyscanner_long['roberta_neu'] = ""
/var/folders/wf/mk3xyd5533b95_xs35xywm480000gn/T/ipykernel_33050/2636168903.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

FileNotFoundError: [Errno 2] No such file or directory: 'your_file.csv'

In [10]:
df = pd.read_csv("skyskanner_roberted.csv")

# Filter the DataFrame to keep only rows where roberta_neg, roberta_pos, and roberta_neu are null
df_filtered = df[df['roberta_neg'].isnull() & df['roberta_pos'].isnull() & df['roberta_neu'].isnull()]
skyscanner_long = df_filtered

skyscanner_long.to_csv("skyskanner_not_roberted")

In [11]:
skyscanner_long = pd.read_csv("skyskanner_not_roberted.csv")

ParserError: Error tokenizing data. C error: Expected 7 fields in line 13, saw 8


In [9]:
length = len(skyscanner_long)
chunk_size = 10
onepercent = (length/100).__floor__()
print(length)
print(onepercent)
currpercent = 1
for i in range(0,length , chunk_size):
    chunk_tokenized = tokenizer(skyscanner_long["review"].iloc[i:i+chunk_size].tolist(), return_tensors='pt',padding='max_length', truncation=True, max_length=514)
    chunk_modeled = model(**chunk_tokenized)
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_neg')] = chunk_modeled.logits[:, 0].detach().numpy()
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_neu')] = chunk_modeled.logits[:, 1].detach().numpy()
    skyscanner_long.iloc[i:i+chunk_size, skyscanner_long.columns.get_loc('roberta_pos')] = chunk_modeled.logits[:, 2].detach().numpy()
    if (i / onepercent) > currpercent:
        print(i/onepercent)
        currpercent+=1

8967
89


IndexError: index out of range in self

In [3]:
skyscanner_long.to_csv('skyskanner_roberted2.csv', index=False)